### Capturing word correlation in input data

In [1]:
import numpy as np
onehots = {}
onehots['cat'] = np.array([1,0,0,0])
onehots['the'] = np.array([0,1,0,0])
onehots['dog'] = np.array([0,0,1,0])
onehots['sat'] = np.array([0,0,0,1])

sentence = ['the','cat','sat']
x = onehots[sentence[0]]+onehots[sentence[1]]+onehots[sentence[2]]

print("Sentence Encoding:" + str(x))

Sentence Encoding:[1 1 0 1]


### Predicitng movie reviews

In [2]:
import sys


In [3]:
# reading the reviews and labels into a list
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

In [4]:
# mapping each review as tokens
tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

In [5]:
"""
creating a set of vocab taking each word from tokens and if len > 0
we add it to the list of words
"""
vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

In [6]:
#enumerating each word to find assign index for each of the words

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

In [7]:
# Converting each sentence to index and then creating set of each token to avoid duplicates

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

In [8]:
# Converting labels to 0 and 1s
target_dataset = list()
for labels in raw_labels:
    if labels == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)